right now, this compares 9 channel CNN vs. GLMs, in terms of raw performance (cc^2 or cc), on ALL neurons (so no subset stuff yet)

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from collections import OrderedDict
from itertools import product
import h5py
import numpy as np
import pandas as pd

In [3]:
from tang_jcompneuro.model_fitting_postprocess import load_model_performance

In [4]:
def check_all():
    subsets_to_check = ('all',
#                         'OT'
                       )
    datasets_to_check = ('MkA_Shape',
#                          'MkE2_Shape'
                        )
    
    models_to_examine = [
        ('cnn', 'b.9'),
        ('cnn', 'b.9_avg'),
        ('cnn', 'b.9_halfsq'),
        ('cnn', 'b.9_avg_halfsq'),
        ('cnn', 'b.9_sq'),
        ('cnn', 'b.9_avg_sq'),
        ('cnn', 'b.9_linear'),
        ('cnn', 'b.9_avg_linear'),
    ] # TODO should make this a ordered dict.
      # so that it will be easier to collect best val model for each model class.
    
    score_all = []
    for dataset, subset in product(datasets_to_check, subsets_to_check):
        
        print(dataset, subset)
        for (model_type, model_subtype) in models_to_examine: 
            score_new_cc = np.asarray(
                [load_model_performance(dataset, subset, 100, s, model_type, model_subtype)['corr'] for s in range(1)]
            ).mean(axis=0)
            
            score_new_cc2 = np.asarray(
                [load_model_performance(dataset, subset, 100, s, model_type, model_subtype)['corr']**2 for s in range(1)]
            ).mean(axis=0)
            
            score_all.append({
                'dataset': dataset,
                'subset': subset,
                'model': model_type + '_' + model_subtype,
                'cc': score_new_cc.mean(),
                'cc2': score_new_cc2.mean(),
                # later on, I can add neuron subset, etc.
            })
            
    score_all = pd.DataFrame(score_all,
                             columns = ['dataset', 'subset', 'model', 'cc', 'cc2']).set_index(['dataset', 'subset', 'model'], verify_integrity=True)
    return score_all

In [5]:
df_all = check_all()

MkA_Shape all


In [6]:
df_all.columns.name = 'metric'

In [7]:
# this is the way to change a multiindex's names.
# df_all.index.rename(['haha','haha2','haha3'], inplace=True)

In [8]:
df_all.unstack('subset')

metric                              cc       cc2
subset                             all       all
dataset   model                                 
MkA_Shape cnn_b.9             0.431644  0.220070
          cnn_b.9_avg         0.415119  0.204510
          cnn_b.9_avg_halfsq  0.412280  0.204517
          cnn_b.9_avg_linear  0.218611  0.062131
          cnn_b.9_avg_sq      0.381723  0.172631
          cnn_b.9_halfsq      0.435210  0.223077
          cnn_b.9_linear      0.401524  0.190735
          cnn_b.9_sq          0.419590  0.206657